In [8]:
import numpy as np
import pandas as ps
from sklearn.cluster import MeanShift

In [64]:
def write_answer(val, name):
    with open(name, "w") as fp:
        fp.write(val)

In [22]:
def import_checkin_foursquare_dat(file_name, save_to):
    with open(file_name, 'r') as fp:
        header = fp.readline()
        header = list(map(str.strip, header.split('|')))[3:5]
        _ = fp.readline()

        cnt = 0
        rows = [header]    
        while len(rows) <= 100001:
            new = fp.readline()
            la, lo = list(map(str.strip, new.split('|')))[3:5]
            cnt += 1 
            
            if la and lo:
                rows.append((la, lo))
            
    with open(save_to, 'w') as fp:
        fp.writelines((','.join(line) + '\n' for line in rows))

In [23]:
# Do this ones to reduce the size of the dataset
import_checkin_foursquare_dat('checkins.dat', 'checkins.csv')

In [24]:
frame = ps.read_csv('checkins.csv')

In [68]:
frame.shape

(100001, 2)

In [26]:
ms = MeanShift(bandwidth=0.1)

In [27]:
ms.fit(frame)

MeanShift(bandwidth=0.1)

In [34]:
result = ms.predict(frame)


In [69]:
cnt = {}
for item in result:
    cnt.setdefault(item, 0)
    cnt[item] += 1


In [70]:
print(len(cnt))
print(len(ms.cluster_centers_))

3231
3231


In [72]:
clusters = {i: cnt[i] for i in cnt if cnt[i] > 15}

In [73]:
centers = [ms.cluster_centers_[c] for c in clusters]

In [75]:
offices = [[33.751277, -118.188740],
           [25.867736, -80.324116],
           [51.503016, -0.075479],
           [52.378894, 4.885084],
           [39.366487, 117.036146],
           [-33.868457, 151.205134]]


In [76]:
from scipy.spatial import distance

In [78]:
selected = None
dist = None
for center in centers:
    new_distance = min(distance.euclidean(center, office) for office in offices)
    if not dist or dist > new_distance:
        selected = center
        dist = new_distance

In [79]:
write_answer(' '.join(str(i) for i in selected), 'results.txt')